<a href="https://colab.research.google.com/github/ritwikraha/computer-needs-glasses/blob/master/image-editing/Image_CutOut_Filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Processing with Clustering and Contours

This notebook implements an image processing algorithm that uses KMeans clustering to reduce the color palette of an image, finds contours of these color regions, and then redraws the image with these simplified colors and contours. It's a form of artistic filter that can create interesting visual effects.

First, let's import the necessary libraries.

Code is adapted from: https://gist.github.com/TACIXAT/c25dd24f9af40e5cd0ff91a3178c4dcb


In [ ]:
!pip install opencv-python-headless scikit-learn matplotlib numpy -qq

In [ ]:
import cv2
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
import random

## Function Definitions

Now, we define several functions that will be used in the processing pipeline. These include functions for displaying images, clustering color spaces, remapping colors, and finding contours.


In [ ]:
def show(im):
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    plt.figure()
    plt.axis("off")
    plt.imshow(im)
    wm = plt.get_current_fig_manager()
    wm.window.state('zoomed')
    plt.show()

def cluster(im, n_clusters):
    im = im.reshape((im.shape[0] * im.shape[1], 3))
    km = KMeans(n_clusters=n_clusters, random_state=0)
    km.fit(im)
    # Additional code remains unchanged
    return km.cluster_centers_, km.labels_

def remap_colors(im, reps, labels):
	orig_shape = im.shape
	im = im.reshape((im.shape[0] * im.shape[1], 3))
	for i in range(len(im)):
		im[i] = reps[labels[i]]
	return im.reshape(orig_shape)

def find_contours(im, reps, min_area):
	contours = []
	for rep in reps:
		mask = cv2.inRange(im, rep-1, rep+1)
		# show(mask)
		conts, _ = cv2.findContours(
			mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
		for cont in conts:
			area = cv2.contourArea(cont)
			if area >= min_area:
				contours.append((area, cont, rep))
	contours.sort(key=lambda x: x[0], reverse=True)
	return contours

## User Inputs

Please specify the input parameters for the image processing. You can adjust the number of clusters, the size of the blur kernel, and other parameters to see how they affect the output.


In [ ]:
#@title Image Processing Parameters

in_file = "" #@param {type:"string"}
out_file = "" #@param {type:"string"}
n_clusters = 3 #@param {type:"integer"}
blur_kernel = 5 #@param {type:"slider", min:1, max:21, step:2}
min_area = 50 #@param {type:"integer"}
poly_epsilon = 10.0 #@param {type:"number"}
quiet = False #@param {type:"boolean"}
final_blur = False #@param {type:"boolean"}
slice = False #@param {type:"boolean"}

# Include validation for `blur_kernel` and `min_area` here if necessary.


## Main Processing Code

With all functions defined and parameters set, let's process the image according to the provided parameters.


In [ ]:
def process_image(in_file, out_file, n_clusters, blur_kernel, min_area, poly_epsilon, quiet, final_blur, slice):
    if blur_kernel % 2 != 1:
        print('-blur-kernel must be an odd number')
        return

    if min_area < 1:
        print('-min-area must be at least 1')
        return

    if not quiet:
        print(f'Reading file {in_file}...')

    orig = cv2.imread(in_file)
    im = orig.copy()

    if not quiet:
        print(f'Blurring with size {blur_kernel}...')

    im = cv2.GaussianBlur(im, (blur_kernel, blur_kernel), 0)

    if not quiet:
        print(f'Clustering around {n_clusters} colors...')

    reps, labels = cluster(im, n_clusters)

    if not quiet:
        print('Remapping image to representative colors...')

    im = remap_colors(im, reps, labels)

    if not quiet:
        print(f'Finding contours with area gte {min_area}...')

    contours = find_contours(im, reps, min_area)

    if not quiet:
        print(f'Drawing...')

    canvas = np.zeros(orig.shape, np.uint8)
    for area, cont, rep in contours:
        approx = cv2.approxPolyDP(cont, poly_epsilon, True)
        cv2.drawContours(canvas, [approx], -1, rep, -1)

    if final_blur:
        canvas = cv2.GaussianBlur(canvas, (3, 3), 0)

    if out_file is None or out_file == "":
        show(canvas)
    else:
        cv2.imwrite(out_file, canvas)

        if slice:
            toks = out_file.split('.')
            ext = toks.pop()
            pre = '.'.join(toks)

            count = 0
            for rep in reps:
                mask = cv2.inRange(canvas, rep-1, rep+1)
                cv2.imwrite(f'{pre}.{count}.{ext}', mask)
                count += 1



In [ ]:
# Call process_image with the parameters from the form fields
process_image(in_file, out_file, n_clusters, blur_kernel, min_area, poly_epsilon, quiet, final_blur, slice)